# Displaying beam based aligmennt measurement re

In [ ]:
import pathlib

from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from pathlib import Path, PosixPath

In [ ]:
import xarray as xr
import numpy as np
import os.path
import matplotlib.pyplot as plt

from dataclasses import dataclass
# %matplotlib notebook
%matplotlib inline

In [ ]:
data_dir = Path(os.getcwd()) / ".." / "bact_analysis_bessyii"/ "scripts"
data_dir = Path(os.path.normpath(data_dir))
data_dir

In [ ]:
os.listdir(data_dir)

In [ ]:
# campain 1788
uids = [
    # variable names without dt ....
    # 'bceb0543-46f4-4bf8-9757-2340cacefb3b',
    # 'b61e48bb-6f7b-4559-9053-f6049fc51bac'
]
uids += [
    # At this run I (pierre) missed to help the script with the bump of out of range.
     # '22a27d85-415b-4a4b-a7ed-fa69d53245b4'.  
    # tune adjusted
    'c6c3ad04-7c4c-4a6d-a7e8-91602cfea726'
]
#campain 1945
uids += [
    "c95ecd6b-ebf6-425a-9a47-8499286cb32f",
    # why do these data not load?
    # "94f42695-4ef5-4659-96c4-de72a4055384",
    "6fa0e851-29b9-4f14-9d1a-d2eb9d47fb80"
]
uids += [
    #campain 1966
     "1e6ec7f3-44a5-4e5c-a798-1d7ce12aafe1",
     "22354cec-864b-4f38-ad46-a9641d07d1ac",
     "eb89753c-5388-4ebb-a992-fc25b478acd8",
    # campain 1976
     "b226943c-1941-46ce-bc35-0530ea6e276c",
     "e0aef7b8-f57e-4594-9618-70d01aaa65a7",
     "e60215ff-62ea-4d3b-a968-f6b0d9d9ee9d",
     "fa22af2e-0398-41eb-94b9-e9b957ba4f31"
]
uids += [
    # campaign 1979
    '69eb8a83-d3bc-42cb-9803-072585124df9',
    '31b041d9-66c6-4906-8be8-9bf9d7c7a789',
    
    # campaign 1982
    '350d945b-6d2f-42b4-892b-7c2c3a9a17d0',
    '9ba454c7-f709-4c42-84b3-410b5ac05d9d'
]
uids += [
    '9e90e2b4-95bf-43a6-9dbd-73a105e28db0',
]
# uids = uids[-7:-4]
uids = uids[-1:]

In [ ]:
uid = uids[-1]
estimated_angles = xr.load_dataset(os.path.join(data_dir, f'estimated_angles_{uid}.nc'))
offsets = xr.load_dataarray(os.path.join(data_dir, f'offsets_{uid}.nc'))

In [ ]:
estimated_angles.orbit

In [ ]:

estimated_angles

In [ ]:
estimated_angles.orbit.attrs['theta']

In [ ]:
estimated_angles.fit_params.sel(name="q1m1d1r", plane="x", result="value", parameter="scaled_angle")

In [ ]:
pscale = 1000

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(
    estimated_angles.ds, estimated_angles.orbit.sel(name="q1m1d1r", plane="x") * pscale
)
ax.set_ylabel("dx [mm]")
ax.set_xlabel("ds [m]")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e6

for uid in uids:
    estimated_angles = xr.load_dataset(os.path.join(data_dir, f'estimated_angles_{uid}.nc'))
    offsets = xr.load_dataarray(os.path.join(data_dir, f'offsets_{uid}.nc'))
    
    ds = estimated_angles.ds_elems
    quad_names = estimated_angles.coords['name'].values
    
    ax_x.errorbar(
        ds,
        offsets.sel(name=quad_names, result="value", plane="x") * a_scale,
        yerr=offsets.sel(name=quad_names, result="error", plane="x") * a_scale,
        fmt=".-",
    )
    ax_y.errorbar(
        ds,
        offsets.sel(name=quad_names, result="value", plane="y") * a_scale,
        yerr=offsets.sel(name=quad_names, result="error", plane="y") * a_scale,
        fmt=".-",
    )
    ax_y.set_xlabel("s [m]")
    ax_x.set_xlabel("s [m]")
    ax_x.set_ylabel("x [$\mu$m]")
    ax_y.set_ylabel("y [$\mu$m]")

In [ ]:
@dataclass
class ReferenceOffsets:
    x: object
    xerr: object
    y: object
    yerr: object
    ds: object

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e6

ref = None
for uid in uids:
    estimated_angles = xr.load_dataset(os.path.join(data_dir, f'estimated_angles_{uid}.nc'))
    offsets = xr.load_dataarray(os.path.join(data_dir, f'offsets_{uid}.nc'))
    
    
    ds = estimated_angles.ds_elems
    quad_names = estimated_angles.coords['name'].values
    x = offsets.sel(name=quad_names, result="value", plane="x") * a_scale
    y = offsets.sel(name=quad_names, result="value", plane="y") * a_scale
    xerr = offsets.sel(name=quad_names, result="error", plane="x") * a_scale
    yerr = offsets.sel(name=quad_names, result="error", plane="x") * a_scale
    if not ref:
        xerr= np.absolute(xerr)
        yerr= np.absolute(yerr)
        ref = ReferenceOffsets(x=x, y=y, xerr=xerr, yerr=yerr, ds=ds)
        print("Set refence plots start after next plot")
        continue 
    else:
        xerr = np.absolute(xerr) + ref.xerr
        yerr = np.absolute(xerr) + ref.yerr
        
    ax_x.errorbar(
        ds,
        x - ref.x,
        yerr=xerr,
        fmt=".-"
    )
    ax_y.errorbar(
        ds,
        y - ref.y,
        yerr=yerr,
        fmt=".-",
    )
    ax_y.set_xlabel("s [m]")
    ax_x.set_xlabel("s [m]")
    ax_x.set_ylabel("x [$\mu$m]")
    ax_y.set_ylabel("y [$\mu$m]")